<a href="https://colab.research.google.com/github/krusegw/cs315/blob/main/RB_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title key_object
#!/usr/bin/env python3
# key_object.py

# Introduction to Algorithms, Fourth edition
# Linda Xiao

#########################################################################
#                                                                       #
# Copyright 2022 Massachusetts Institute of Technology                  #
#                                                                       #
# Permission is hereby granted, free of charge, to any person obtaining #
# a copy of this software and associated documentation files (the       #
# "Software"), to deal in the Software without restriction, including   #
# without limitation the rights to use, copy, modify, merge, publish,   #
# distribute, sublicense, and/or sell copies of the Software, and to    #
# permit persons to whom the Software is furnished to do so, subject to #
# the following conditions:                                             #
#                                                                       #
# The above copyright notice and this permission notice shall be        #
# included in all copies or substantial portions of the Software.       #
#                                                                       #
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,       #
# EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF    #
# MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND                 #
# NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS   #
# BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN    #
# ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN     #
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE      #
# SOFTWARE.                                                             #
#                                                                       #
#########################################################################

class KeyObject:
	"""Used for testing anything that requires a key."""

	def __init__(self, string, key):
		self.string = string
		self.key = key

	@staticmethod
	def get_key(x):
		return x.key

	@staticmethod
	def set_key(x, key):
		x.key = key

	def __gt__(self, obj2):
		return self.key > obj2.key

	def __str__(self):
		return self.string

In [2]:
# @title binary_search_tree
#!/usr/bin/env python3
# binary_search_tree.py

# Introduction to Algorithms, Fourth edition
# Linda Xiao and Tom Cormen

#########################################################################
#                                                                       #
# Copyright 2022 Massachusetts Institute of Technology                  #
#                                                                       #
# Permission is hereby granted, free of charge, to any person obtaining #
# a copy of this software and associated documentation files (the       #
# "Software"), to deal in the Software without restriction, including   #
# without limitation the rights to use, copy, modify, merge, publish,   #
# distribute, sublicense, and/or sell copies of the Software, and to    #
# permit persons to whom the Software is furnished to do so, subject to #
# the following conditions:                                             #
#                                                                       #
# The above copyright notice and this permission notice shall be        #
# included in all copies or substantial portions of the Software.       #
#                                                                       #
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,       #
# EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF    #
# MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND                 #
# NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS   #
# BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN    #
# ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN     #
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE      #
# SOFTWARE.                                                             #
#                                                                       #
#########################################################################

class BinarySearchTreeNode:

	def __init__(self, data):
		"""Initialize all instance variables of node to None."""
		self.left = None
		self.right = None
		self.parent = None
		self.data = data

	def __str__(self):
		"""Print data."""
		return str(self.data)

class BinarySearchTree:
	"""Binary tree properties:
	1. left child key <= parent key
	2. right child key >= parent key
	"""

	def __init__(self, get_key_func=None, nil=None):
		"""Initialize binary search tree.

		Arguments:
		get_key_func -- an optional function that returns the key for the
		objects stored. May be a static function in the object class. If
		omitted, then identity function is used.
		nil -- sentinel value for the parent of the root and children of the
		leaves of the tree. Optional parameter that is given when implementing
		subclasses of BinarySearchTree.
		"""
		if get_key_func is None:
			self.get_key = lambda x: x
		else:
			self.get_key = get_key_func

		if nil is None:
			self.nil = BinarySearchTreeNode(None)
		else:
			self.nil = nil 	  # may be RedBlackTreeNode, ...
		self.root = self.nil  # empty tree

	def get_root(self):
		"""Return root."""
		return self.root

	def inorder_tree_walk(self, x, func=print):
		"""Run a function on all the nodes of the subtree rooted at node x in an inorder tree walk.

		Arguments:
		x -- root of the subtree
		func -- function to run on each node in the subtree.  If omitted, print.
		"""
		# Run func on the key of the root in between visiting the left and right subtrees.
		if x != self.nil:
			self.inorder_tree_walk(x.left, func)
			func(x)
			self.inorder_tree_walk(x.right, func)

	def search(self, x, k):
		"""Return a node with a given key k in the subtree rooted at x, or self.nil if no node with key k exists."""
		if x == self.nil or k == self.get_key(x.data):
			return x
		elif k < self.get_key(x.data):
			return self.search(x.left, k)   # if present, k must be in left subtree
		else:
			return self.search(x.right, k)  # if present, k must be in right subtree

	def iterative_search(self, x, k):
		"""Return a node with a given key k in the subtree rooted at x, or self.nil if no node with key k exists."""
		while x != self.nil and k != self.get_key(x.data):
			if k < self.get_key(x.data):
				x = x.left   # if present, k must be in the left subtree
			else:
				x = x.right  # if present, k must be in the right subtree
		return x

	def minimum(self, x):
		"""Return a node in subtree rooted at x with the smallest key."""
		while x.left != self.nil:
			x = x.left  # keep going left
		return x        # leftmost key

	def maximum(self, x):
		"""Return a node in subtree rooted at x with the largest key."""
		while x.right != self.nil:
			x = x.right  # keep going right
		return x         # rightmost key

	def successor(self, x):
		"""Return the node in the subtree rooted at x with the smallest key greater than x's key."""
		if x.right != self.nil:
			return self.minimum(x.right)  # leftmost node in right subtree
		# Find the lowest ancestor of x whose left child is also an ancestor of x.
		y = x.parent
		while y != self.nil and x == y.right:  # continue up the tree
			x = y
			y = y.parent
		return y

	def predecessor(self, x):
		"""Return the node in the subtree rooted at x with the greatest key less than x's key."""
		if x.left != self.nil:
			return self.maximum(x.left)  # rightmost node in left subtree
		# Find the lowest ancestor of x whose right child is also an ancestor of x.
		y = x.parent
		while y != self.nil and x == y.left:  # continue up the tree
			x = y
			y = y.parent
		return y

	def tree_insert(self, data):
		"""Initialize a node with data and insert into this binary search tree."""
		z = BinarySearchTreeNode(data)
		# Initialize node's left and right child pointers with defined nil values.
		z.right = self.nil
		z.left = self.nil

		self.tree_insert_node(z)

	# Helper method for inserting a node.  Assumes that node z is already created and initialized.
	def tree_insert_node(self, z):
		"""Insert node z into this binary search tree."""
		x = self.root         # node being compared with z
		y = self.nil          # y will be the parent of z
		while x != self.nil:  # descend until reaching a leaf
			y = x
			if self.get_key(z.data) < self.get_key(x.data):
				x = x.left
			else:
				x = x.right
		z.parent = y  # found the location -- insert z with parent y
		if y == self.nil:
			self.root = z  # tree was empty
		elif self.get_key(z.data) < self.get_key(y.data):
			y.left = z     # assign z as y's left child
		else:
			y.right = z    # assign z as y's right child

	def transplant(self, u, v):
		"""Replace the subtree rooted at node u with the subtree rooted at node v."""
		if u.parent == self.nil:
			self.root = v         # replacing the root
		elif u == u.parent.left:  # is u a left child?
			u.parent.left = v     # update left child to v
		else:                     # u is a right child
			u.parent.right = v    # update right child to v

		# Differ from the textbook because here we have a self.nil value as a node, so
		# that methods can be used for subclasses of BST.  No need to test whether v
		# has a parent--it does, even if v is self.nil.
		v.parent = u.parent       # update v's parent

	def tree_delete(self, z):
		"""Delete node z from this binary search tree and maintain the binary search tree property.

		Assumption:
		Node z appears in this binary search tree.
		"""
		if z is None or z == self.nil:
			raise RuntimeError("Cannot delete sentinel or None node.")

		if z.left == self.nil:
			self.transplant(z, z.right)      # replace z by its right child
		elif z.right == self.nil:
			self.transplant(z, z.left)       # replace z by its left child
		else:
			y = self.minimum(z.right)        # y is z's successor
			if y != z.right:                 # is y farther down the tree?
				self.transplant(y, y.right)  # replace y by its right child
				y.right = z.right            # z's right child becomes y's right child
				y.right.parent = y
			self.transplant(z, y)            # replace z by its successor y
			y.left = z.left                  # and give z's left child to y, which had no left child
			y.left.parent = y

	def is_BST(self, x=None):
		"""Return a boolean indicating whether this tree obeys the binary search tree properties.

		Argument:
		x -- root of a subtree.  None indicates root of the entire BST.
		"""
		if x is None:
			x = self.root
		if x == self.nil:
			return True                    # for an empty BST
		if x.left is not self.nil:         # check the left subtree
			if self.get_key(x.left.data) > self.get_key(x.data):
				return False               # left child's key > x's key
			elif not self.is_BST(x.left):  # check the rest of the left subtree
				return False
		if x.right is not self.nil:        # check the right subtree
			if self.get_key(x.right.data) < self.get_key(x.data):
				return False               # right child's key < x's key
			elif not self.is_BST(x.right):
				return False               # check the rest of the right subtree

		return True                        # no error found in the subtree rooted at x

	def pretty_print(self, node, depth=0):
		"""Return a string representing a subtree of a binary search tree with nodes of the same depth in the same column.
		If you tilt your head to the left, it should look like the nodes are positioned correctly.

		Arguments:
		node -- root of a subtree to print
		depth -- depth of the node within the binary search tree
		"""
		result = ""
		if node == self.nil:
			return result
		# Print the right subtree.  Printing right before left so that the BST looks correct with head tilted.
		if node.right != self.nil:
			result += self.pretty_print(node.right, depth + 1)
		result += ('  ' * depth) + str(node) + '\n'  # print this node
		# Print the left subtree.
		if node.left != self.nil:
			result += self.pretty_print(node.left, depth + 1)
		return result

	def __str__(self):
		"""Return a string representing a binary search tree with nodes of the same depth in the same column.
		If you tilt your head to the left, it should look like the nodes are positioned correctly."""
		return self.pretty_print(self.root)

In [4]:
# @title  red_black_tree
#!/usr/bin/env python3
# red_black_tree.py

# Introduction to Algorithms, Fourth edition
# Linda Xiao and Tom Cormen

#########################################################################
#                                                                       #
# Copyright 2022 Massachusetts Institute of Technology                  #
#                                                                       #
# Permission is hereby granted, free of charge, to any person obtaining #
# a copy of this software and associated documentation files (the       #
# "Software"), to deal in the Software without restriction, including   #
# without limitation the rights to use, copy, modify, merge, publish,   #
# distribute, sublicense, and/or sell copies of the Software, and to    #
# permit persons to whom the Software is furnished to do so, subject to #
# the following conditions:                                             #
#                                                                       #
# The above copyright notice and this permission notice shall be        #
# included in all copies or substantial portions of the Software.       #
#                                                                       #
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,       #
# EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF    #
# MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND                 #
# NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS   #
# BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN    #
# ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN     #
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE      #
# SOFTWARE.                                                             #
#                                                                       #
#########################################################################

class RedBlackTreeNode(BinarySearchTreeNode):

	def __init__(self, data):
		"""Initialize this BinarySearchTreeNode and add a color attribute."""
		BinarySearchTreeNode.__init__(self, data)
		self.is_red = True

	def __str__(self):
		"""Return <data>: <color>."""
		return BinarySearchTreeNode.__str__(self) + ": " + ("red" if self.is_red else "black")


class RedBlackTree(BinarySearchTree):
	""" Red-black tree properties:
	Binary search tree tree plus
	1. Every node is either red or black.
	2. The root is black.
	3. Every leaf is black.
	4. If a node is red, then both its children are black.
	5. For each node, all simple paths from the node to descendant leaves
	contain the same number of black nodes.
	"""

	def __init__(self, get_key_func=None, nil=None):
		"""Initialize this RedBlackTree with asentinel that is a RedBlackTreeNode.

		Arguments:
		get_key_func -- an optional function that returns the key for the
		objects stored. May be a static function in the object class. If
		omitted, then identity function is used.
		nil -- sentinel value for the parent of the root and children of the
		leaves of the tree. Optional parameter that is given when implementing
		subclasses of RedBlackTree.
		"""
		if nil is None:
			self.nil = RedBlackTreeNode(None)
		else:
			self.nil = nil  # may be IntervalTreeNode, ...
		BinarySearchTree.__init__(self, get_key_func, self.nil)
		self.nil.is_red = False  # the root must be black

	def left_rotate(self, x):
		"""Make the right child of x become the new root of the subtree with a left rotation.

		Assumptions:
		x has a right child.
		The root's parent is self.nil.
		"""
		y = x.right
		x.right = y.left            # turn y's left subtree into x's right subtree
		if y.left != self.nil:      # if y's left subtree is not empty ...
			y.left.parent = x       # ... then x becomes the parent of the subtree's root
		y.parent = x.parent         # x's parent becomes y's parent
		if x.parent == self.nil:    # if x was the root ...
			self.root = y           # ... then y becomes the root
		elif x == x.parent.left:    # otherwise, if x was a left child ...
			x.parent.left = y       # ... then y becomes a left child
		else:
			x.parent.right = y      # otherwise, x was a right child, and now y is
		y.left = x                  # make x become y's left child
		x.parent = y

	def right_rotate(self, x):
		"""Make the left child of x become the new root of the subtree with a right rotation.

		Assumptions:
		x has a left child.
		The root's parent is self.nil.
		"""
		y = x.left
		x.left = y.right            # turn y's right subtree into x's left subtree
		if y.right != self.nil:     # if y's right subtree is not empty ...
			y.right.parent = x      # ... then x becomes the parent of the subtree's root
		y.parent = x.parent         # x's parent becomes y's parent
		if x.parent == self.nil:    # if x was the root ...
			self.root = y           # ... then y becomes the root
		elif x == x.parent.right:   # otherwise, if x was a right child ...
			x.parent.right = y      # ... then y becomes a left child
		else:
			x.parent.left = y       # otherwise, x was a left child, and now y is
		y.right = x                 # make x become y's right child
		x.parent = y

	def tree_insert(self, data):
		"""Insert a new node with data and maintain the red-black tree."""
		self.insert_node(RedBlackTreeNode(data))  # defaults to red

	def insert_node(self, z):
		"""Insert a new node z and maintain the red-black tree."""
		# Initialize z's left and right with defined nil values to maintain proper tree structure.
		z.right = self.nil
		z.left = self.nil

		BinarySearchTree.tree_insert_node(self, z)  # insert z into the tree
		self.rb_insert_fixup(z)   # correct any violations of red-black properties

	def rb_insert_fixup(self, z):
		"""If the red-black properties are violated after node z was inserted, restore them."""
		while z.parent.is_red:                      # two reds in a row
			if z.parent == z.parent.parent.left:    # is z's parent a left child?
				y = z.parent.parent.right           # y is z's uncle
				if y.is_red:
					# Case 1: z's uncle y is red.
					z.parent.is_red = False         # color z's parent black
					y.is_red = False                # color z's uncle black
					z.parent.parent.is_red = True   # color z's grandparent red
					z = z.parent.parent             # continue from z's grandparent
				else:  # z's uncle y is black
					if z == z.parent.right:
						# Case 2: z is a right child
						z = z.parent
						self.left_rotate(z)         # z is now a left child
					# Case 3: z is a left child
					z.parent.is_red = False
					z.parent.parent.is_red = True
					self.right_rotate(z.parent.parent)  # no consecutive reds
			else:  # z's parent is a right child
				y = z.parent.parent.left            # y is z's uncle
				if y.is_red:
					# Case 1: z's uncle y is red.
					z.parent.is_red = False         # color z's parent black
					y.is_red = False                # color z's uncle black
					z.parent.parent.is_red = True   # color z's grandparent red
					z = z.parent.parent             # continue from z's grandparent
				else:  # z's uncle y is black
					if z == z.parent.left:
						# Case 2: z is a left child
						z = z.parent
						self.right_rotate(z)        # z is now a right child
					# Case 3: z is a right child
					z.parent.is_red = False
					z.parent.parent.is_red = True
					self.left_rotate(z.parent.parent)  # no consecutive reds
		self.root.is_red = False                    # restore the root as black

	# Unlike the textbook, there is no need to modify the implementation of transplant
	# from the BinarySearchTree class.  It already tests whether u.parent == self.nil,
	# and it already assigns v.parent = u.parent unconditionally.

	def tree_delete(self, z):
		"""Delete z and maintain the red-black tree.

		Assumption:
		Node z exists in the tree.
		"""
		if z is None or z == self.nil:
			raise RuntimeError("Cannot delete sentinel or None node.")

		# y is the node either removed from the tree or moved within the tree.
		y = z
		y_original_color = y.is_red
		if z.left == self.nil:           # z has no left child
			x = z.right                  # x will move into y's position
			self.transplant(z, z.right)  # replace z by its right child
		elif z.right == self.nil:        # z has a left child, but no right child
			x = z.left                   # x will move into y's position.
			self.transplant(z, z.left)   # replace z by its left child
		else: 	                         # two children
			y = self.minimum(z.right)    # y is z's successor
			y_original_color = y.is_red
			x = y.right
			if y != z.right:             # is y farther down the tree?
				self.transplant(y, y.right)  # replace y by its right child
				y.right = z.right        # z's right child becomes y's right child
				y.right.parent = y
			else:
				x.parent = y             # in case x is self.nil
			self.transplant(z, y)        # replace z by its successor y
			y.left = z.left              # and give z's left child to y, which had no left child
			y.left.parent = y
			y.is_red = z.is_red          # give y same color as z
		if not y_original_color:
			# If any red-black violations occurred, correct them.
			self.rb_delete_fixup(x)

	def rb_delete_fixup(self, x):
		"""Maintain the red-black properties after deletion.

		Argument:
		x -- the node that moved into the deleted position
		"""
		while x != self.root and not x.is_red:
			# x is "doubly black."
			if x == x.parent.left:              # is x a left child?
				w = x.parent.right              # w is x's sibling
				if w.is_red:
					# Case 1: x's sibling w is red.
					# Because w is red, w's parent--which is x's parent--must be black.
					# Switch the colors of w and x's parent.
					w.is_red = False
					x.parent.is_red = True
					self.left_rotate(x.parent)
					w = x.parent.right          # transform to case 2, 3, or 4
				if not w.left.is_red and not w.right.is_red:
					# Case 2: x's sibling w is black, and both of w's children are black.
					# Remove one black from x and w.
					w.is_red = True
					x = x.parent                # the extra black moves up one level
				else:
					# Case 3: x's sibling w is black, w's left child is red,
					# and w's right child is black.
					if not w.right.is_red:
						# Switch the colors of w and its left child w.left.
						w.left.is_red = False
						w.is_red = True
						self.right_rotate(w)
						w = x.parent.right      # transform to case 4
					# Case 4: x's sibling w is black, and w's right child is red.
					# Color changes and a left rotation on x.parent allow the extra black on x to vanish.
					w.is_red = x.parent.is_red
					x.parent.is_red = False
					w.right.is_red = False
					self.left_rotate(x.parent)
					x = self.root               # the loop terminates upon the next condition test
			else:  # x is a right child
				w = x.parent.left               # w is x's sibling
				if w.is_red:
					# Case 1: x's sibling w is red.
					# Because w is red, w's parent--which is x's parent--must be black.
					# Switch the colors of w and x's parent.
					w.is_red = False
					x.parent.is_red = True
					self.right_rotate(x.parent)
					w = x.parent.left           # transform to case 2, 3, or 4
				if not w.right.is_red and not w.left.is_red:
					# Case 2: x's sibling w is black, and both of w's children are black.
					# Remove one black from x and w.
					w.is_red = True
					x = x.parent                # the extra black moves up one level
				else:
					# Case 3: x's sibling w is black, w's left child is red,
					# and w's right child is black.
					if not w.left.is_red:
						# Switch the colors of w and its right child w.right.
						w.right.is_red = False
						w.is_red = True
						self.left_rotate(w)
						w = x.parent.left       # transform to case 4
					# Case 4: x's sibling w is black, and w's right child is red.
					# Color changes and a right rotation on x.parent allow the extra black on x to vanish.
					w.is_red = x.parent.is_red
					x.parent.is_red = False
					w.left.is_red = False
					self.right_rotate(x.parent)
					x = self.root               # the loop terminates upon the next condition test
		x.is_red = False                        # ensure that the root is black

	def is_rb_tree(self):
		"""Return a boolean indicating whether the tree is a legal red-black tree."""
		# Return false if not a binary search tree.
		if not BinarySearchTree.is_BST(self):
			return False
		# The root and sentinel should be black.
		if self.root.is_red or self.nil.is_red:
			return False

		return self.is_rb_subtree(self.root) != -1

	def is_rb_subtree(self, x):
		"""Determines whether a subtree of a red-black tree has valid black-heights and no two red nodes in a row.

		Argument:
		x -- root of the subtree

		Returns:
		-1 for invalid red black tree
		black height for valid red black tree
		"""
		if x == self.nil:
			return 0
		if x.is_red and x.parent.is_red:
			return -1  # two red nodes in a row
		left_black_height = self.is_rb_subtree(x.left)
		right_black_height = self.is_rb_subtree(x.right)
		# The black-heights of the left and right subtrees should be equal.
		if left_black_height == -1 or right_black_height == -1 or left_black_height != right_black_height:
			return -1
		else:
			# Increment the black-height if x is black.
			return left_black_height + int(not x.is_red)

In [6]:
# @title testing RB Tree code
# Testing
if __name__ == "__main__":

	import numpy as np

	# Insert.
	rb_tree1 = RedBlackTree()
	array1 = np.arange(0, 100, 13)
	np.random.shuffle(array1)
	for value in array1:
		rb_tree1.tree_insert(value)
	print(rb_tree1.is_rb_tree())
	print(rb_tree1)
	rb_tree1.inorder_tree_walk(rb_tree1.get_root())
	# Search.
	node39 = rb_tree1.search(rb_tree1.get_root(), 39)
	print("Found: " + str(node39))
	# Unsuccessful search.
	print("Found: " + str(rb_tree1.search(rb_tree1.get_root(), 55)))
	# Iterative.
	print("Found: "
		+ str(rb_tree1.iterative_search(rb_tree1.get_root(), 39)))
	print("Found: "
		+ str(rb_tree1.iterative_search(rb_tree1.get_root(), 55)))
	# Minimum and maximum.
	print("Max: " + str(rb_tree1.maximum(rb_tree1.get_root())))
	print("Min: " + str(rb_tree1.minimum(rb_tree1.get_root())))
	# Delete.
	rb_tree1.tree_delete(node39)
	print("After deleting 39: ")
	print(rb_tree1.is_rb_tree())
	print(rb_tree1)
	node52 = rb_tree1.search(rb_tree1.get_root(), 52)
	rb_tree1.tree_delete(node52)
	print("After deleting 52: ")
	print(rb_tree1.is_rb_tree())
	print(rb_tree1)

	rb_tree1.inorder_tree_walk(rb_tree1.get_root())
	# Delete a node that does not exist.
	node99 = rb_tree1.search(rb_tree1.get_root(), 99)
	try:
		rb_tree1.tree_delete(node99)
	except RuntimeError as e:
		print(e)
	print(rb_tree1.is_rb_tree())
	print(rb_tree1)

	# Tree with objects.
	rb_tree2 = RedBlackTree(KeyObject.get_key)
	states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "HI", "NH", "NY"]
	list2 = []
	for i in range(len(states)):
		list2.append(KeyObject(states[i], i))
	array2 = np.array(list2)
	np.random.shuffle(array2)
	for x in array2:
		rb_tree2.tree_insert(x)
	print(rb_tree2.is_rb_tree())
	print(rb_tree2)
	nodeCO = rb_tree2.search(rb_tree2.get_root(), 5)
	rb_tree2.tree_delete(nodeCO)
	print("After deleting CO:")
	print(rb_tree2.is_rb_tree())
	print(rb_tree2)
	# Check that is_rb_tree works correctly by making a black node turn red.
	for x in array2:
		# Find a non-root black node.
		node = rb_tree2.search(rb_tree2.get_root(), KeyObject.get_key(x))
		if not node.is_red and node != rb_tree2.get_root():
			break
	node.is_red = True
	print(rb_tree2.is_rb_tree())
	# Restore blackness to that node, but make two reds in a row.
	node.is_red = False
	for x in array2:
		node = rb_tree2.search(rb_tree2.get_root(), KeyObject.get_key(x))
		if node.is_red:
			break
	node.parent.is_red = True
	print(rb_tree2.is_rb_tree())
	# Restore blackness, but make the root red.
	node.parent.is_red = False
	rb_tree2.get_root().is_red = True
	print(rb_tree2.is_rb_tree())
	# Make the root black but the sentinel red.
	rb_tree2.get_root().is_red = False
	rb_tree2.nil.is_red = True
	print(rb_tree2.is_rb_tree())

	# Exhaustive testing.
	rb_tree3 = RedBlackTree()
	array2 = np.arange(-100, 1000)
	np.random.shuffle(array2)
	for value in array2:
		rb_tree3.tree_insert(value)  # Covers every insert fixup case.
		if not rb_tree3.is_rb_tree():
			print(rb_tree3)
			break
	print(rb_tree3.is_rb_tree())

	np.random.shuffle(array2)
	for value in array2:
		to_delete = rb_tree3.search(rb_tree3.get_root(), value)
		rb_tree3.tree_delete(to_delete)
		if not rb_tree3.is_rb_tree():
			print(rb_tree3)
			break
	print(rb_tree3.is_rb_tree())

True
    91: red
  78: black
    65: red
52: black
    39: black
  26: red
      13: red
    0: black

0: black
13: red
26: red
39: black
52: black
65: red
78: black
91: red
Found: 39: black
Found: None: black
Found: 39: black
Found: None: black
Max: 91: red
Min: 0: black
After deleting 39: 
True
    91: red
  78: black
    65: red
52: black
    26: black
  13: red
    0: black

After deleting 52: 
True
    91: red
  78: black
65: black
    26: black
  13: red
    0: black

0: black
13: red
26: black
65: black
78: black
91: red
Cannot delete sentinel or None node.
True
    91: red
  78: black
65: black
    26: black
  13: red
    0: black

True
    NY: black
      NH: red
  HI: red
    CT: black
      CO: red
CA: black
    AR: black
  AZ: red
    AK: black
      AL: red

After deleting CO:
True
    NY: black
      NH: red
  HI: red
    CT: black
CA: black
    AR: black
  AZ: red
    AK: black
      AL: red

False
False
False
False
True
True
